In [ ]:
import csv
import os
import numpy as np
from keras.preprocessing import image
from keras.applications.xception import Xception
from keras.models import load_model
from tensorflow.python.keras.utils import get_file

In [ ]:
# Path to the directory containing the face images
face_images_dir = "path/to/face/images/directory"

# Download the Xception model weights if not already available
xception_weights_path = get_file(
    'xception_weights.h5',
    'https://github.com/fchollet/deep-learning-models/releases/download/v0.4/xception_weights.h5',
    cache_subdir='models',
    file_hash='b0042744bf5b25fce3cb969f33bebb97')

# Path to the LQ model (EDSR)
edsr_model_path = "path/to/edsr_model.h5"

# Load the HQ model (Xception)
hq_model = Xception(weights=xception_weights_path, include_top=False)

# Load the LQ model (EDSR)
lq_model = load_model(edsr_model_path)

# Initialize the predictions dictionary
predictions = {}

# Iterate over the face images
for filename in os.listdir(face_images_dir):
    if filename.endswith(".jpg"):
        image_path = os.path.join(face_images_dir, filename)
        img = image.load_img(image_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = x / 255.0  # Normalize the image

        # Make predictions using the HQ model (Xception)
        hq_prediction = hq_model.predict(x)[0][0]

        # Make predictions using the LQ model (EDSR)
        lq_prediction = lq_model.predict(x)[0][0]

        # Store the predictions in the dictionary
        predictions[filename] = [hq_prediction, lq_prediction]

# Update the sample sheet with the predicted probabilities
sample_sheet = "path/to/sample_sheet.csv"

with open(sample_sheet, 'r') as file:
    csv_data = list(csv.reader(file))

# Find the index of the "predict" column
predict_index = csv_data[0].index("predict")

# Update the predicted probabilities in the sample sheet
for row in csv_data[1:]:
    filename = row[0]
    if filename in predictions:
        hq_prediction, lq_prediction = predictions[filename]
        row[predict_index] = f"HQ: {hq_prediction:.4f}, LQ: {lq_prediction:.4f}"

# Write the updated sample sheet with predicted probabilities
with open(sample_sheet, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(csv_data)